<a href="https://colab.research.google.com/github/pallabjyotidk/PyTorch-Free-Code-Camp/blob/main/Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to Linear Regression

In this tutorial, we'll discuss one of the foundational algorithms in machine learning: Linear regression. We'll create a model that predicts crop yields for apples and oranges (target variables) by looking at the average temperature, rainfall, and humidity (input variables or features) in a region.

In [ ]:
import numpy as np
import torch

In [ ]:
# Inputs are: temp, rainfall, humidity

inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102,43,37],
                   [69,96,70]], dtype = 'float32')

In [ ]:
# Targets are: apples, oranges

targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [ ]:
# Convert inputs and targets to tensors

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


The weights and biases `(w11, w12,... w23, b1 & b2)` can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [ ]:
# Weights and Biases

w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.3272, -0.9931, -0.8553],
        [-0.6031, -0.7307,  0.7519]], requires_grad=True)
tensor([-0.5350,  0.2470], requires_grad=True)


Our model should compute:
  `X x W' + b` (where W' indicates W transpose)

In [ ]:
def model(x):
  return x.matmul(w.t()) + b
  # same as return x @ w.t() + b

In [ ]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[-127.7376,  -60.4024],
        [-172.4438,  -70.8125],
        [-211.6869, -106.5228],
        [-108.2605,  -64.8669],
        [-178.3215,  -58.8788]], grad_fn=<AddBackward0>)


In [ ]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Loss Function
We will use MSE Loss here.
* Calculate the difference between two matrices `preds` and `targets`
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of elemets in the resulting matrix.

In [ ]:
# MSE Loss
def mse(t1, t2):
  diff = t1 - t2
  return torch.sum(diff * diff)/diff.numel()

`torch.sum` returns the sum of all the elements in a tensor. The `.numel` method of a tensor returns the number of elements in a tensor.

In [ ]:
# Compute Loss
loss = mse(preds, targets)
print(loss)

tensor(44936.9883, grad_fn=<DivBackward0>)


## Compute Gradients
With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`.
The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [ ]:
# Compute Gradients
loss.backward()
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.3272, -0.9931, -0.8553],
        [-0.6031, -0.7307,  0.7519]], requires_grad=True)
tensor([[-19588.7500, -22306.7168, -13562.6211],
        [-13731.3711, -15481.8311,  -9343.2432]])


## Adjust weights and biases to reduce the loss
The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's slope w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will increase the loss
* **decreasing** the weight element's value slightly will decrease the loss

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will decrease the loss
* **decreasing** the weight element's value slightly will increase the loss.

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of the *gradient descent optimization* algorithm that we'll use to improve our model (by **descending along the gradient**).


In [ ]:
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [ ]:
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

## Train the model using gradient descent
As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can *train* the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero



In [ ]:
# Generate predictions
preds = model(inputs)

In [ ]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(30543.0410, grad_fn=<DivBackward0>)


In [ ]:
# Compute Gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-15964.5332, -18405.4336, -11156.7021],
        [-11209.2852, -12767.9355,  -7669.4580]])
tensor([-192.8822, -134.3706])


In [ ]:
# Adjust weights and reset gradients
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5

# Reset gradients to zero
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

## Train for multiple epochs

In [ ]:
# Train for 100 epochs
for i in range(100):
  preds = model(inputs)
  loss = mse(preds, targets)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [ ]:
# Calculate Loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(248.0520, grad_fn=<DivBackward0>)


## Linear Regression using PyTorch Built-ins

In [ ]:
import torch.nn as nn

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

## Dataset and DataLoader
We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
# Define Dataset
train_ds =TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements from `1st` to `3rd` row. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
# Define Data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [ ]:
for xb, yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[102.,  43.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  67.,  43.],
        [103.,  43.,  36.],
        [ 91.,  88.,  64.]])
tensor([[ 22.,  37.],
        [104., 118.],
        [ 56.,  70.],
        [ 20.,  38.],
        [ 81., 101.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear
Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [ ]:
# Define Model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3816, -0.3297,  0.0636],
        [ 0.5067,  0.2913,  0.5445]], requires_grad=True)
Parameter containing:
tensor([ 0.1066, -0.0643], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [ ]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.3816, -0.3297,  0.0636],
         [ 0.5067,  0.2913,  0.5445]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1066, -0.0643], requires_grad=True)]

In [ ]:
# Generate Predictions
preds = model(inputs)
preds

tensor([[-47.1063,  79.8521],
        [-59.5640, 106.5235],
        [-73.5872, 114.6299],
        [-50.6390,  84.2873],
        [-53.4263, 100.9741],
        [-47.1581,  80.0675],
        [-59.1707, 106.7766],
        [-73.9051, 115.6811],
        [-50.5872,  84.0719],
        [-52.9812, 101.0119],
        [-46.7130,  80.1053],
        [-59.6158, 106.7388],
        [-73.9805, 114.3767],
        [-51.0841,  84.2495],
        [-53.3745, 100.7587]], grad_fn=<AddmmBackward>)

## Loss Function
Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [ ]:
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities.

In [ ]:
# Define loss function
loss_fn = F.mse_loss

In [ ]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(9993.7129, grad_fn=<MseLossBackward>)


## Optimizer
Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term *stochastic* indicates that samples are selected in random batches instead of as a single group.

In [ ]:
# Define OPtimizer
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model
We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [ ]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):

  for epoch in range(num_epochs):
    for xb, yb in train_dl:
      # Generate predictions
      pred = model(xb)

      # Calculate loss
      loss = loss_fn(pred, yb)

      # Compute gradients
      loss.backward()

      # Update parameters using gradients
      opt.step()

      # Reset gradients to zero
      opt.zero_grad()
    
    if (epoch+1) % 10 == 0:
       print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [ ]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 322.0905
Epoch [20/100], Loss: 276.0595
Epoch [30/100], Loss: 266.3441
Epoch [40/100], Loss: 122.9957
Epoch [50/100], Loss: 99.7929
Epoch [60/100], Loss: 46.3808
Epoch [70/100], Loss: 72.7059
Epoch [80/100], Loss: 16.3982
Epoch [90/100], Loss: 47.3540
Epoch [100/100], Loss: 13.1727


In [ ]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.9696,  71.6711],
        [ 81.9134,  99.8957],
        [116.6362, 132.4541],
        [ 26.7020,  44.8881],
        [ 98.2553, 113.1881],
        [ 56.8733,  70.7521],
        [ 81.6914,  99.7870],
        [116.9292, 132.9947],
        [ 27.7984,  45.8071],
        [ 99.1296, 113.9984],
        [ 57.7476,  71.5624],
        [ 80.8171,  98.9767],
        [116.8583, 132.5628],
        [ 25.8277,  44.0778],
        [ 99.3517, 114.1071]], grad_fn=<AddmmBackward>)

In [ ]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])